In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.manifold import TSNE
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.metrics import r2_score
import random
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/myanimelist-dataset/user_anime000000000032.csv
/kaggle/input/myanimelist-dataset/user_anime000000000000.csv
/kaggle/input/myanimelist-dataset/user_anime000000000029.csv
/kaggle/input/myanimelist-dataset/user_anime000000000019.csv
/kaggle/input/myanimelist-dataset/user_anime000000000011.csv
/kaggle/input/myanimelist-dataset/user_anime000000000025.csv
/kaggle/input/myanimelist-dataset/user_anime000000000040.csv
/kaggle/input/myanimelist-dataset/user_anime000000000047.csv
/kaggle/input/myanimelist-dataset/user.csv
/kaggle/input/myanimelist-dataset/user_anime000000000002.csv
/kaggle/input/myanimelist-dataset/user_anime000000000009.csv
/kaggle/input/myanimelist-dataset/user_anime000000000033.csv
/kaggle/input/myanimelist-dataset/user_anime000000000006.csv
/kaggle/input/myanimelist-dataset/user_anime000000000068.csv
/kaggle/input/myanimelist-dataset/user_anime000000000034.csv
/kaggle/input/myanimelist-dataset/user_anime000000000061.csv
/kaggle/input/myanimelist-dataset/user_ani

****Collaborative filtering****

considerations -  
- extremely high number of users

In [4]:
anime  = pd.read_csv('../input/myanimelist-dataset/anime.csv', sep='\t')
anime_anime = pd.read_csv('../input/myanimelist-dataset/anime_anime.csv', sep='\t')
user = pd.read_csv('../input/myanimelist-dataset/user.csv', sep='\t')
user_user = pd.read_csv('../input/myanimelist-dataset/user_user.csv', sep='\t')
user['serial_index'] = list(user.index)
user = user.set_index('user_id')
anime['serial_index'] = list(anime.index)
anime = anime.set_index('anime_id')

In [5]:
files_to_skip = ['user.csv','anime.csv','user_user.csv','anime_anime.csv']
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename in files_to_skip:
            continue
        files.append(os.path.join(dirname, filename))

In [ ]:
# df = None
# for file in files:  
#     print(file)
#     df_temp = pd.read_csv(file,sep='\t')[['user_id', 'anime_id', 'score','last_interaction_date']]
#     df_temp = df_temp[~df_temp['score'].isnull()]
#     if df is None:
#         df = df_temp
#     else:
#         df = pd.concat([df,df_temp])

In [5]:
#user anime interaction loader
class User_anime_data:
    def __init__(self,file_list,user_ids,anime_ids,batch_size=32,shuffle=True):
        self.user_ids = user_ids
        self.anime_ids = anime_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_list = file_list
        
    def get_gen(self):
        if self.shuffle:
            random.shuffle(self.file_list)
        for file in self.file_list:
            if isinstance(file, bytes):
                file = file.decode('utf-8')
            print(file)
            df_file = pd.read_csv(str(file), sep='\t')[['user_id','anime_id','score']]
            df_file = df_file[~df_file['score'].isnull()]
            print(len(df_file))
            if self.shuffle:
                df_file = df_file.sample(frac=1)
            prev_i = 0
            for i in range(self.batch_size,len(df_file),self.batch_size):
                df_minibatch = df_file[prev_i:i]
                users_minibatch = np.array(self.user_ids.loc[df_minibatch['user_id'].tolist()])
                anime_minibatch = np.array(self.anime_ids.loc[df_minibatch['anime_id'].tolist()])
                y = np.array(df_minibatch['score'])
                X = np.stack((users_minibatch,anime_minibatch),axis=1)

                yield X,y
        
        
# def user_anime_data(file_list,user_ids,anime_ids,batch_size=32,shuffle=True):
#     if shuffle:
#         random.shuffle(file_list)
#     for file in file_list:
#         if isinstance(file, bytes):
#             file = file.decode('utf-8')
#         print(file)
#         df_file = pd.read_csv(str(file), sep='\t')[['user_id','anime_id','score']]
#         df_file = df_file[~df_file['score'].isnull()]
#         if shuffle:
#             df_file = df_file.sample(frac=1)
#         prev_i = 0
#         for i in range(batch_size,len(df_file),batch_size):
#             df_minibatch = df_file[prev_i:i]
#             users_minibatch = np.array(user_ids.loc[df_minibatch['user_id'].tolist()])
#             anime_minibatch = np.array(anime_ids.loc[df_minibatch['anime_id'].tolist()])
#             y = np.array(df_minibatch['score'])
#             X = np.stack((users_minibatch,anime_minibatch),axis=1)
            
#             yield X,y
            


In [15]:
#Simple collaborative model
def collaborative_model(num_users, num_movies, emb_dim):
    x = tf.keras.Input((2))
    x_user = tf.keras.layers.Embedding(num_users,emb_dim,
#                                        embeddings_regularizer=tf.keras.regularizers.L2(1e-4),
                                       name='users')(x[:,0])
    x_movie = tf.keras.layers.Embedding(num_movies,emb_dim,
#                                        embeddings_regularizer=tf.keras.regularizers.L2(1e-4),
                                       name='movies')(x[:,1])
    out = tf.linalg.diag_part(tf.linalg.matmul(x_user,x_movie,transpose_b=True))
    
    return tf.keras.Model(inputs=x, outputs=out, name="colab_model")

In [22]:
def sample_gen():
#     X = np.random.randint(0,1000,size=(13284744,2))
#     y = np.random.randint(0,10,size=(13284744))
    X  = tf.ones((13284744,2))
    y = tf.ones((13284744))
    prev_i = 0
    for i in range(32,X.shape[0],32):
        yield X[prev_i:i,:], y[prev_i:i]

gen = sample_gen()
    
    

model = collaborative_model(len(user),len(anime),8)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse'])

model.fit(gen,
callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=4,
    restore_best_weights=True
)])

    981/Unknown - 598s 610ms/step - loss: 0.0241 - mse: 0.1414

KeyboardInterrupt: 

In [7]:
#Create a TF dataset. Due to huge number of users, extremely memory hungry. Need approximation methods
gen = User_anime_data(files,user_ids=user['serial_index'],anime_ids=anime['serial_index'])
# dataset = tf.data.Dataset.from_generator(gen.get_gen, 
#           output_signature=(
#                             tf.TensorSpec(shape=(None,2), dtype=tf.int32),
#                             tf.TensorSpec(shape=(None), dtype=tf.int32)))
dataset = gen.get_gen()

model = collaborative_model(len(user),len(anime),8)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse'])

model.fit(dataset,
callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=4,
    restore_best_weights=True
)])    
    
umat = model.get_layer('users').get_weights()[0]
mmat = model.get_layer('movies').get_weights()[0]


2022-11-06 18:16:17.688891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 18:16:17.693559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 18:16:17.694598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 18:16:17.696449: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

/kaggle/input/myanimelist-dataset/user_anime000000000003.csv
1819971


2022-11-06 18:16:26.957879: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


   1291/Unknown - 36s 27ms/step - loss: 29.8999 - mse: 37.4480

2022-11-06 18:17:13.366963: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.37GiB (rounded to 6837305344)requested by op gradient_tape/colab_model/tf.linalg.diag_part/zeros_like
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-06 18:17:13.367025: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2022-11-06 18:17:13.367050: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 32, Chunks in use: 31. 8.0KiB allocated for chunks. 7.8KiB in use in bin. 156B client-requested in use in bin.
2022-11-06 18:17:13.367065: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
20

ResourceExhaustedError:  OOM when allocating tensor with shape[41344,41344] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/colab_model/tf.linalg.diag_part/zeros_like (defined at tmp/ipykernel_24/258032227.py:19) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_696]

Function call stack:
train_function


In [ ]:
dataset = tf.data.Dataset.from_generator(gen.get_gen, 
          output_signature=(
                            tf.TensorSpec(shape=(None,2), dtype=tf.int32),
                            tf.TensorSpec(shape=(None), dtype=tf.int32))
                                        )

for i,data in enumerate(dataset):
#     print(i)
    if not i%1000:
        print(i)
    

2022-11-05 21:56:33.320705: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-11-05 21:56:33.441474: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


/kaggle/input/myanimelist-dataset/user_anime000000000015.csv
1977198
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


In [8]:
mmat.shape

(13379, 8)